In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('/content/df_finn(충북제외).csv', encoding='cp949', index_col=0)

# feature importance 기반 클러스터링
x,y 좌표 뿐만 아니라 인구수, 주점개수, 도로환경을 활용해 거점 찾기

In [ ]:
new_df = df[['x','y','중분류_인구수','중분류_주점개수','도로환경']]
new_df

,x,y,중분류_인구수,중분류_주점개수,도로환경
0,127.298872,36.489101,21461,0,83.95
1,127.058056,36.923056,333602,281,72.59
2,127.382337,36.336437,470374,122,75.88
3,127.128056,36.787778,657821,420,70.30
4,127.057063,36.774519,333602,281,72.59
...,...,...,...,...,...
100070,127.273611,36.435000,8699,9,83.95
100071,127.376396,36.350271,470374,122,75.88
100072,127.370828,36.309233,470374,122,75.88
100073,127.221604,36.773741,657821,420,70.30


In [ ]:
new_df

,x,y,중분류_인구수,중분류_주점개수,도로환경
0,127.298872,36.489101,21461,0,83.95
1,127.058056,36.923056,333602,281,72.59
2,127.382337,36.336437,470374,122,75.88
3,127.128056,36.787778,657821,420,70.30
4,127.057063,36.774519,333602,281,72.59
...,...,...,...,...,...
100070,127.273611,36.435000,8699,9,83.95
100071,127.376396,36.350271,470374,122,75.88
100072,127.370828,36.309233,470374,122,75.88
100073,127.221604,36.773741,657821,420,70.30


# K-means Clustering
15개 중심 찾기

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters=15)
kmeans.fit(new_df)
labels = kmeans.labels_

centers = kmeans.cluster_centers_

In [ ]:
cluster_df = pd.DataFrame(centers, columns = new_df.columns)
cluster_df

,x,y,중분류_인구수,중분류_주점개수,도로환경
0,127.019161,36.806665,333602.000000,281.000000,72.590000
1,127.150623,36.815879,657821.000000,420.000000,70.300000
2,126.689868,36.907370,167955.000000,160.000000,48.150000
3,127.373870,36.336881,470374.000000,122.000000,75.880000
4,127.187920,36.491794,29861.932363,3.790405,84.415678
5,126.819358,36.463361,99673.072453,91.494473,81.208809
6,127.409308,36.325735,227108.000000,132.000000,78.880000
7,127.343923,36.368324,356093.000000,136.000000,78.670000
8,126.518596,36.535555,61769.066225,57.735099,75.940464
9,127.265003,36.523566,8929.366728,2.532169,83.950000


In [ ]:
cluster_df.to_csv('cluster_df.csv', encoding='utf-8-sig')

In [ ]:
!pip install folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import folium

In [ ]:
cluster_df['x']

0     127.019161
1     127.150623
2     126.689868
3     127.373870
4     127.187920
5     126.819358
6     127.409308
7     127.343923
8     126.518596
9     127.265003
10    127.112145
11    126.974570
12    126.785092
13    127.176894
14    127.444529
Name: x, dtype: float64

In [ ]:
accident_coordinates = []
for i in range(len(cluster_df)):
    accident_coordinates.append([cluster_df['y'][i], cluster_df['x'][i]])

In [ ]:
accident_coordinates

[[36.80666454764111, 127.01916053106271],
 [36.815878918412615, 127.15062260534751],
 [36.90737010898843, 126.68986808182709],
 [36.336881162633446, 127.37386981395099],
 [36.49179435316555, 127.18792019874164],
 [36.46336081085586, 126.81935763715661],
 [36.325735331069005, 127.40930773957535],
 [36.36832382563935, 127.3439225030116],
 [36.535554893782624, 126.51859606081028],
 [36.52356594800551, 127.26500277247243],
 [36.185641217799805, 127.11214500679233],
 [36.5792850150702, 126.9745697497097],
 [36.68556943484045, 126.78509206522445],
 [36.30780006295829, 127.17689378889428],
 [36.332239441864196, 127.44452923653537]]

# 클러스터 중심 시각화

In [ ]:
m = folium.Map(location=[36.489101,127.298872], zoom_start=9)

# 주점
for i in accident_coordinates:
    folium.Marker(location=i).add_to(m)

m.save('cluster.html')
m